# Syft Keras দিয়ে প্রাইভেট প্রেডিকশান (Private Predictions with Syft Keras)

# পর্ব ১: পাবলিক ট্রেইনিং (Public Training)

এই টিউটোরিয়ালে স্বাগতম! নিম্নলিখিত নোটবুকগুলিতে আপনি কীভাবে প্রাইভেট প্রেডিকশান করবেন তা শিখবেন। প্রাইভেট প্রেডিকশানের অর্থ হল পুরো প্রক্রিয়া জুড়ে ডেটা অনবরত এনক্রিপ্টেড হতে থাকবে। প্রক্রিয়ার কোন পর্যায়েই ব্যবহারকারি প্রকৃত ডেটা প্রকাশ করেনা, শুধুমাত্র এনক্রিপ্টেড (গোপনে শেয়ারকৃত) ডেটা প্রকাশ করে। প্রাইভেট প্রেডিকশান সরবরাহ করার জন্য Syft Keras [TF Encrypted](https://github.com/tf-encrypted/tf-encrypted) নামের একটি লাইব্রেরি ব্যবহার করে। TF Encrypted অত্যাধুনিক ক্রিপ্টোগ্রাফি ও মেশিন লার্নিং প্রযুক্তির সমন্বয়ে তৈরি, কিন্তু আপনাকে এসব নিয়ে চিন্তা করতে হবেনা। আপনি শুধুমাত্র মেশিন লার্নিং অ্যাপ্লিকেশনটিতে ফোকাস করতে পারেন।

আপনি কেবলমাত্র তিনটি পদক্ষেপের সাহায্যে প্রাইভেট প্রেডিকশান করা শুরু করতে পারেন:
- **পর্ব  ১**: আপনার মডেলটিকে সাধারণ Keras দিয়ে ট্রেইন করুন।
- **পর্ব  ২**: আপনার মেশিন লার্নিং মডেলটিকে সুরক্ষিত করে সার্ভ (সার্ভার) করুন।
- **পর্ব  ৩**: সুরক্ষিত মডেলটি হতে কুয়েরি (query) করে প্রাইভেট প্রেডিকশান গ্রহণ করুন।

ঠিক আছে, আসুন এই তিনটি ধাপ অনুসরণ করে আপনি কীভাবে ব্যবহারকারীর গোপনীয়তা বা মডেল সুরক্ষা ত্যাগ না করে কার্যকর মেশিন লার্নিং পরিষেবা স্থাপন করতে পারেন তা দেখি।

লেখক:
- Jason Mancuso - Twitter: [@jvmancuso](https://twitter.com/jvmancuso)
- Yann Dupis - Twitter: [@YannDupis](https://twitter.com/YannDupis)
- Morten Dahl - Twitter: [@mortendahlcs](https://github.com/mortendahlcs)

সমর্থনে:
- Dropout Labs - Twitter: [@dropoutlabs](https://twitter.com/dropoutlabs)
- TF Encrypted - Twitter: [@tf_encrypted](https://twitter.com/tf_encrypted)

অনুবাদক:

- Sourav Das - Twitter: [@adventuroussrv](https://twitter.com/adventuroussrv)
- Zarreen Reza - Twitter: [@zarreennreza](https://twitter.com/zarreennreza)

## Keras এর সাহায্যে মডেল ট্রেইন করুন (Train your model with Keras)

আপনার প্রোজেক্টের জন্য গোপনীয়তা-সুরক্ষিত মেশিন লার্নিং প্রযুক্তি ব্যবহার করার জন্য আপনার নতুন কোন মেশিন লার্নিং ফ্রেমওয়ার্ক শেখার প্রয়োজন নেই। আপনার যদি Keras সম্পর্কে সাধারণ জ্ঞান থাকে, তবেই আপনি Syft Keras দিয়ে এই প্রযুক্তি ব্যবহার করতে পারেন। আপনি যদি পূর্বে কখনো Keras ব্যবহার না করে থাকেন, তবে এই [Keras documentation](https://keras.io) থেকে এর সম্পর্কে জানতে পারেন। 

প্রাইভেট প্রেডিকশান প্রদান করার আগে প্রথম ধাপ হচ্ছে সাধারণ Keras দিয়ে আপনার মডেল ট্রেইন করা। উদাহরনস্বরূপ, এখানে আমরা হস্ত-লিখিত সংখ্যা শ্রেণীকরণের জন্য একটি মডেল ট্রেইন করব। মডেল  ট্রেইন করার জন্য আমরা বহুল-ব্যবহৃত [MNIST](http://yann.lecun.com/exdb/mnist/) ডেটাসেট ব্যবহার করব।

[এই উদাহরণটি](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py) আমরা Keras রেপোসিটরি থেকে ধার নিয়েছি। আপনার মডেল ট্রেইন করার জন্য নিচের সেলগুলো রান করুন।

In [ ]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Activation

batch_size = 128
num_classes = 10
epochs = 2

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## ভবিষ্যতে প্রাইভেট প্রেডিকশান করার জন্য মডেলের ওয়েট (weights) গুলো সংরক্ষন করুন (Save your model's weights for future private prediction)

চমৎকার! আপনার মডেল ট্রেইন হয়ে গিয়েছে। আসুন model.save() ব্যবহার করে মডেলের ওয়েট (weights) গুলো সংরক্ষন করি। পরবর্তী নোটবুকে আমরা Syft Keras এ এই ওয়েট (weights) গুলো লোড করে প্রাইভেট প্রেডিকশান সার্ভ করা শুরু করব।  

In [ ]:
model.save('short-conv-mnist.h5')